In [1]:
import tensorflow as tf
import numpy as np
import pickle

In [2]:
data = pickle.load(open('../data/data.npz', 'rb'))

In [3]:
# # print(np.array(data['inputs'][15]).shape)
# print(data['inputs'][15].shape)
# print(data['units'])

(82, 15, 1816)
1816


In [4]:
class layer(object):
    def __init__(self):
        raise NotImplementedError
    
    def __str__(self):
        raise NotImplementedError
    
class affine_layer(layer):
    def __init__(self, name, inputs, shape, activation, reuse=False):
        with tf.variable_scope('affine', reuse=reuse):
            self.weights = tf.get_variable(name = name+'_weights', initializer=tf.truncated_normal(shape, stddev=0.05), dtype=tf.float32)
            self.bias = tf.get_variable(name = name+'_bias', initializer=tf.zeros([shape[-1]]))
            self.outputs = activation(tf.add(tf.matmul(inputs, self.weights), self.bias))
    def __str__(self):
        return self.outputs.__str__()
    
class conv_layer(layer):
    def __init__(self, name, inputs, shape, activation, reuse=False):
        with tf.variable_scope('conv', reuse=reuse):
            self.kernel = tf.get_variable(name = name+'_kernel', initializer=tf.truncated_normal(shape, stddev=0.05), dtype=tf.float32)
            self.bias = tf.get_variable(name = name+'_bias', initializer=tf.zeros([shape[-1]]))
            self.outputs = activation(tf.add(tf.nn.conv2d(inputs, self.kernel, padding='VALID', strides=[1,1,1,1]), self.bias))
    def __str__(self):
        return self.outputs.__str__()
    
class pooling_layer(layer):
    def __init__(self, name, inputs, reuse=False):
        with tf.variable_scope('pooling', reuse=reuse):
            self.outputs = tf.nn.max_pool(name = name+'_maxpooling', value=inputs, ksize=[1,2,2,1], padding='VALID', strides=[1,2,2,1])
    def __str__(self):
        return self.outputs.__str__()
        
class reshape_layer(layer):
    def __init__(self, name, inputs, shape, reuse=False):
        with tf.variable_scope('reshape', reuse=reuse):
            self.outputs = tf.reshape(name=name, tensor=inputs, shape=shape)
    def __str__(self):
        return self.outputs.__str__()
        
class deconv_layer(layer):
    def __init__(self, name, inputs, kernel_shape, output_shape, activation, reuse=False):
        with tf.variable_scope('deconv', reuse=reuse):
            self.kernel = tf.get_variable(name = name+'_kernel', initializer=tf.truncated_normal(kernel_shape, stddev=0.05), dtype=tf.float32)
            self.bias = tf.get_variable(name = name+'_bias', initializer=tf.zeros([kernel_shape[-2]]))
            self.outputs = activation(tf.add(tf.nn.conv2d_transpose(inputs, self.kernel, output_shape=output_shape, padding='VALID', strides=[1,2,2,1]), self.bias))
    def __str__(self):
        return self.outputs.__str__()
    
class lstm_layer(layer):
    def __init__(self, name, inputs, n_units, reuse=False):
        with tf.variable_scope('lstm', reuse=reuse):
            self.outputs, self.states = tf.nn.dynamic_rnn(
                cell = tf.contrib.rnn.BasicLSTMCell(n_units),
                inputs = inputs,
                dtype = tf.float32)
    def __str__(self):
        return self.outputs.__str__()

In [5]:
train_inputs = data['inputs']
train_targets = data['targets']
batch_sizes = data['batch_size']
n_units = data['units']
n_hidden = 300

In [41]:
graph = tf.Graph()
with graph.as_default():
    
    input_placeholder = tf.placeholder(tf.float32, [None, None, n_units])
    target_placeholder = tf.placeholder(tf.float32, [None, None, n_units])
    batch_size_placeholder = tf.placeholder(tf.int32)
    step_size_placeholder = tf.placeholder(tf.int32)
    
    
    reshape1 = reshape_layer('reshape_1', input_placeholder, [-1, n_units])
    affine1 = affine_layer('affine_1', reshape1.outputs, [n_units, n_hidden], tf.nn.relu)
    reshape2 = reshape_layer('reshape_2', affine1.outputs, [batch_size_placeholder, step_size_placeholder, n_hidden])
    
    
    lstm1 = lstm_layer('lstm_1', reshape2.outputs, n_hidden, False)
    reshape3 = reshape_layer('reshape_3', lstm1.outputs, [-1, n_hidden])
    affine2 = affine_layer('affine_2', reshape3.outputs, [n_hidden, n_units], tf.identity)
    reshape4 = reshape_layer('reshape_4', affine2.outputs, [batch_size_placeholder, step_size_placeholder, n_units])
    
#     affine_layer_2 = affine_layer('affine_2', affine_layer_1.outputs, [200, n_units], tf.identity)

    output_layer = reshape4
    
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=output_layer.outputs, labels=target_placeholder))
    optimizer = tf.train.AdamOptimizer(1e-3).minimize(loss)
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(write_version=tf.train.SaverDef.V1)
    
    for i in range(100):
        errs = 0.0
        c = 0
        for key in batch_sizes.keys():

            input_batch = train_inputs[key]
            target_batch = train_targets[key]
#             print(key, input_batch.shape)
            step_size = key
            batch_size = input_batch.shape[0]
            
            feed_dict = {step_size_placeholder:step_size, batch_size_placeholder:batch_size, input_placeholder:input_batch, target_placeholder:target_batch}
            _, err = sess.run([optimizer, loss], feed_dict=feed_dict)
            
            errs += err
            c += 1
        saver.save(sess, '../models/model.ckpt', global_step = i)
        print(errs / c)

0.299996203658
0.0313206675716
0.00954300266775
0.00622999741498
0.00540882302448
0.00502968973226
0.00480251447439
0.00464799282649
0.0045343148381
0.00444613621486
0.00437529682831
0.00431679643578
0.00426738410765
0.00422503439371
0.00418699022636
0.00414837237843
0.00411066631645
0.0040794666786
0.00405289368196


0.00402958124566
0.00400911944664
0.0039910743521
0.00397490894168
0.00396038263782
0.00394705455807
0.00393493552551
0.00392388631449
0.00391380803547
0.00390464607908
0.00389609108397
0.00388819132367
0.00388096134658
0.00387420390987
0.00386788264961
0.00386111327503
0.00385499747724
0.00384982554665
0.00384503708108


0.00384046670021
0.00383623637203
0.00383225862275
0.00382852650276
0.00382493904114
0.00382155527812
0.003818339132
0.00381523330527
0.00381233541041
0.00380956225633
0.00380691764594
0.00380442683783
0.00380204519201
0.00379975427958
0.0037975767839
0.00379548270509
0.00379345165989


KeyboardInterrupt: 

In [ ]:
with graph.as_default():
    
    input_placeholder = tf.placeholder(tf.float32, [None, None, n_units])
    target_placeholder = tf.placeholder(tf.float32, [None, None, n_units])
    batch_size_placeholder = tf.placeholder(tf.int32)
    step_size_placeholder = tf.placeholder(tf.int32)
    
    
    reshape1 = reshape_layer('reshape_1', input_placeholder, [-1, n_units])
    affine1 = affine_layer('affine_1', reshape1.outputs, [n_units, n_hidden], tf.nn.relu)
    reshape2 = reshape_layer('reshape_2', affine1.outputs, [batch_size_placeholder, step_size_placeholder, n_hidden])
    
    
    lstm1 = lstm_layer('lstm_1', reshape2.outputs, n_hidden, False)
    reshape3 = reshape_layer('reshape_3', lstm1.outputs, [-1, n_hidden])
    affine2 = affine_layer('affine_2', reshape3.outputs, [n_hidden, n_units], tf.identity)
    reshape4 = reshape_layer('reshape_4', affine2.outputs, [batch_size_placeholder, step_size_placeholder, n_units])
    
#     affine_layer_2 = affine_layer('affine_2', affine_layer_1.outputs, [200, n_units], tf.identity)

    output_layer = reshape4
    
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=output_layer.outputs, labels=target_placeholder))
    optimizer = tf.train.AdamOptimizer(1e-3).minimize(loss)
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(write_version=tf.train.SaverDef.V1)
    

    input_batch = train_inputs[key]
    target_batch = train_targets[key]
#             print(key, input_batch.shape)
    step_size = key
    batch_size = input_batch.shape[0]
            
    feed_dict = {step_size_placeholder:step_size, batch_size_placeholder:batch_size, input_placeholder:input_batch, target_placeholder:target_batch}
    err = sess.run([loss], feed_dict=feed_dict)
            
            errs += err
            c += 1
        saver.save(sess, '../models/model.ckpt', global_step = i)
        print(errs / c)